In [58]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import numpy as np 
import pandas as pd

In [60]:
dataset_path = "./dataset.csv"

df = pd.read_csv(
    dataset_path,
    # dtype is specified to prevent a DtypeWarning
    dtype={
        "full_query": str,
        "label": int,
        "statement_type": str,
        "query_template_id": str,
        "attack_payload": str,
        "attack_id": str,
        "attack_technique": str,
        "attack_desc": str,
        "split": str,
        "sqlmap_status" : str,
        "attack_stage" : str,
        "tamper_method" : str

    },
)

display(df)


,full_query,label,attack_payload,attack_desc,attack_stage,tamper_method,sqlmap_status,statement_type,query_template_id,attack_id,attack_technique,split
0,"SELECT a.id, a.name, a.municipality, a.iso_cou...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S10,NaN,NaN,test
1,"INSERT INTO runways (airport_ref, airport_iden...",1,NaN,NaN,recon,space2comment,NaN,insert,airport-I9,error-7,error,test
2,"UPDATE airport SET wikipedia_link = ""https://e...",1,NaN,NaN,recon,equaltolike,NaN,update,airport-U3,error-6,error,test
3,SHOW GLOBAL STATUS,0,NaN,NaN,NaN,NaN,NaN,admin,airport-admin5,NaN,NaN,test
4,"DELETE FROM navaids WHERE ident = ""GS'||(SELEC...",1,NaN,NaN,recon,sleep2getlock,NaN,delete,airport-D6,error-1,error,test
...,...,...,...,...,...,...,...,...,...,...,...,...
36879,"SELECT a.id, a.ident, a.name, a.latitude_deg, ...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S14,NaN,NaN,test
36880,"SELECT a.id, a.ident, a.name, a.latitude_deg, ...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S14,NaN,NaN,test
36881,"SELECT a.id, a.ident, a.name, a.latitude_deg, ...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S14,NaN,NaN,test
36882,"SELECT a.id, a.ident, a.name, a.latitude_deg, ...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S14,NaN,NaN,test


In [61]:
def display_attack_normal_per_set(df : pd.DataFrame):
    df_train_atk = df[(df["label"] == 1) & (df["split"] == "train")]  
    df_train_normal = df[(df["label"] == 0) & (df["split"] == "train")]  

    df_test_atk = df[(df["label"] == 1) & (df["split"] == "test")]  
    df_test_normal = df[(df["label"] == 0) & (df["split"] == "test")]  

    print(f"Number of attacks in train: {df_train_atk.shape[0]}")
    print(f"Number of normal in train: {df_train_normal.shape[0]}")
    print(f"Number of attacks in test: {df_test_atk.shape[0]}")
    print(f"Number of normal in test: {df_test_normal.shape[0]}")

display_attack_normal_per_set(df)

Number of attacks in train: 4116
Number of normal in train: 4309
Number of attacks in test: 2369
Number of normal in test: 26090


In [62]:
def display_ids_per_set(df: pd.DataFrame):
    df_train = df[df["split"] == "train"]
    df_test = df[df["split"] == "test"]

    tids_train = df_train["query_template_id"].unique()
    tids_test = df_test["query_template_id"].unique()
    tids_complement = set(tids_test) - set(tids_train)
    print(f"Template IDS in df train: {len(tids_train)}, {tids_train}")
    print(f"Template IDS in df test:   {len(tids_test)}, {tids_test}")
    print(
        f"Template IDs in train but NOT in test: {len(tids_complement)}, {sorted(tids_complement)}"
    )

    # Compute which template only have normal queries
    df_n = df.loc[df["label"] == 0,"query_template_id"].unique()
    df_a = df.loc[df["label"] == 1,"query_template_id"].unique()

    print(f"Templates IDs with no attacks: {set(df_n) - set(df_a)}")




display_ids_per_set(df)

Template IDS in df train: 22, ['airport-S10' 'airport-S11' 'airport-D6' 'airport-S8' 'airport-I9'
 'airport-I5' 'airport-U3' 'airport-admin14' 'airport-admin9'
 'airport-admin8' 'airport-admin3' 'airport-admin13' 'airport-D7'
 'airport-admin11' 'airport-admin4' 'airport-admin5' 'airport-admin1'
 'airport-admin12' 'airport-admin7' 'airport-admin10' 'airport-admin6'
 'airport-admin2']
Template IDS in df test:   24, ['airport-S10' 'airport-I9' 'airport-U3' 'airport-admin5' 'airport-D6'
 'airport-S8' 'airport-I4' 'airport-S11' 'airport-S14' 'airport-I5'
 'airport-D7' 'airport-admin9' 'airport-admin14' 'airport-admin11'
 'airport-admin13' 'airport-admin3' 'airport-admin6' 'airport-admin1'
 'airport-admin2' 'airport-admin4' 'airport-admin7' 'airport-admin8'
 'airport-admin12' 'airport-admin10']
Template IDs in train but NOT in test: 2, ['airport-I4', 'airport-S14']
Templates IDs with no attacks: {'airport-admin6', 'airport-admin3', 'airport-admin11', 'airport-admin10', 'airport-admin2', 'air

In [64]:
def _stmt_proportion(df: pd.DataFrame):
    return df['statement_type'].value_counts(normalize=True)

def display_ratio_per_stmt(df : pd.DataFrame):
    
    df_a = df[df["label"] == 1]
    df_n = df[df["label"] == 0]
    print(f"Proportion amongst normal: {_stmt_proportion(df=df_n)}")
    print(f"Proportion amongst attacks: {_stmt_proportion(df=df_a)}")
        
    df_train = df[df["split"] == "train"]
    df_test = df[df["split"] == "test"]
    print(f"Proportion amongst train: {_stmt_proportion(df=df_train)}")
    print(f"Proportion amongst test: {_stmt_proportion(df=df_test)}")

display_ratio_per_stmt(df=df)

Proportion amongst normal: statement_type
select    0.861245
insert    0.105628
delete    0.015066
admin     0.010263
update    0.007796
Name: proportion, dtype: float64
Proportion amongst attacks: statement_type
insert    0.524904
select    0.366230
update    0.073863
delete    0.035004
Name: proportion, dtype: float64
Proportion amongst train: statement_type
select    0.582671
insert    0.275727
update    0.059466
delete    0.055905
admin     0.026231
Name: proportion, dtype: float64
Proportion amongst test: statement_type
select    0.830915
insert    0.150813
update    0.007555
delete    0.007520
admin     0.003198
Name: proportion, dtype: float64
